<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages and data

In [2]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [65]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

Import GPS data

In [3]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
file_path = 'data/GPS 2018-2023_NoContact.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [4]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
file_path = 'data/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

Import wellbeing data


In [ ]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint values

The sprint values are separated in different rows per session,

we want to merge in into a single row per session.

## Transform data before merging sprint rows

In [5]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [6]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [8]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [7]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

In [10]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55390 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


## Merge sprint values

Print values to verify later

In [8]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each session

In [9]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [10]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

In [11]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [15]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [16]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [12]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of both dataframes

In [13]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [14]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   object        
 9

Verify class imbalances

In [15]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7798
Injury: 37
%: 0.47%


# Merge df with 'weight' in wellbeing dataframe

In [ ]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [ ]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [ ]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [ ]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [16]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [17]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

In [18]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


In [19]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [20]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


In [21]:
df_merged[df_merged['% Max Speed'] == 0]

,DATE,Column2,PLAYER,Injury,season,LEAGUE,preseason-season,MANAGER,Total D,>19.8,> 25 Km/h,ACC,DEC,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
3268,2022-09-12,M-1,37990,0.0,2022-2023,CHAMPIONSHIP,SEASON,EDWARDS,5215.0,549.0,50.0,51.0,18.0,37990.0,FULL BACK,32.174,0.0,74.792,2022-2023,37.28,22.669514,0.0,-100.0
5185,2022-04-02,MD,65042,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,9.4,7.6,65042.0,CENTRE MIDFIELDER,23.592,0.0,5.000,2021-2022,34.92,20.048756,0.0,-100.0
5187,2022-04-02,MD,23085,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,3.6,6.2,23085.0,CENTRE MIDFIELDER,17.784,0.0,5.000,2021-2022,30.73,20.278653,0.0,-100.0
5191,2022-04-02,MD,17316,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,6.2,5.6,17316.0,WINGER,21.586,0.0,5.000,2021-2022,33.73,20.441103,0.0,-100.0
5195,2022-04-02,MD,42579,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,23.0,12.6,42579.0,CENTRE BACK,21.274,0.0,5.000,2021-2022,29.76,19.989098,0.0,-100.0
5199,2022-04-02,MD,23081,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,3.6,6.2,23081.0,CENTRE BACK,17.784,0.0,5.000,2021-2022,34.94,20.537766,0.0,-100.0
5201,2022-04-02,MD,37709,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,14.8,6.8,37709.0,CENTRE BACK,21.886,0.0,5.000,2021-2022,36.56,22.374467,0.0,-100.0
6701,2021-11-24,M-5,10452,0.0,2021-2022,PREMIER LEAGUE,SEASON,RANIERI,2354.4,0.0,0.0,18.8,18.0,10452.0,FULL BACK,13.458,0.0,15.000,2021-2022,34.54,22.291234,0.0,-100.0
6831,2021-11-07,MD,21079,0.0,2021-2022,PREMIER LEAGUE,SEASON,RANIERI,10554.0,734.0,361.0,95.0,108.0,21079.0,WINGER,35.094,75.0,100.450,2021-2022,36.54,21.962986,0.0,-100.0
7400,2022-10-19,MD (AWAY),77765,0.0,2022-2023,CHAMPIONSHIP,SEASON,BILIC,1646.0,77.0,1.0,13.0,23.0,77765.0,CENTRE MIDFIELDER,20.330,2.0,18.000,2022-2023,32.08,21.291520,0.0,-100.0


In [22]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

In [23]:
df_merged[(df_merged['PLAYER'] == 65042) & (df_merged['DATE'] == '2022-04-02')]

,DATE,Column2,PLAYER,Injury,season,LEAGUE,preseason-season,MANAGER,Total D,>19.8,> 25 Km/h,ACC,DEC,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
5185,2022-04-02,MD,65042,0.0,2021-2022,PREMIER LEAGUE,SEASON,HODGSON,800.0,600.0,0.0,9.4,7.6,65042.0,CENTRE MIDFIELDER,23.592,0.0,5.0,2021-2022,34.92,20.048756,67.560137,17.673139


# Reduce data volume/size

Drop rows from preseason since its not the same intensity as regular season


In [ ]:
# df_merged = df_merged[df_merged['preseason-season'] != 'PRESEASON']

# df_merged.reset_index(drop=True, inplace=True)

Class imbalance

In [ ]:
# print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
# print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
# print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

## Players who have never been injured

In [65]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

# df_filtered = df_merged.copy()

In [66]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 4686
Injury: 37
%: 0.78%


Verify at least one player had an injury

In [67]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,2.0
12086.0,1.0
17316.0,2.0
18096.0,3.0
20083.0,1.0
21079.0,2.0
23081.0,2.0
23085.0,2.0


# Change column names

In [68]:
column_rename_dict = {
    'Column2': 'Microcycle',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': '>19.8',
    '> 25 Km/h': '>25',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': '>19.8_Rel',
    '> 25 Km/h_Rel': '>25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4723 entries, 0 to 7802
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4723 non-null   object        
 2   PLAYER              4723 non-null   int64         
 3   Injury              4723 non-null   float64       
 4   season              4723 non-null   object        
 5   LEAGUE              4723 non-null   object        
 6   preseason-season    4723 non-null   object        
 7   MANAGER             4723 non-null   object        
 8   TD                  4723 non-null   float64       
 9   >19.8               4723 non-null   float64       
 10  >25                 4723 non-null   float64       
 11  ACC                 4723 non-null   float64       
 12  DEC                 4723 non-null   float64       
 13  PlayerID            4723 non-null   float64       
 1

# Drop unnecesary columns

In [69]:
df_filtered = df_filtered.drop(columns=['LEAGUE', 'MANAGER', 'PLAYER', 'Season'])

# Calculate relative values

We want the max and avg values for players that played over 85 in a Match Day

Remove spaces in 'Microcycle' column

In [70]:
df_filtered['Microcycle'] = df_filtered['Microcycle'].str.replace(' ', '', regex=False)
df_filtered["Microcycle"].unique()

array(['M+2', 'M+1', 'MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M-4',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

In [71]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    microcycle_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Microcycle'].isin(microcycle_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [72]:
metrics_rel = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']

#df_rel = calculate_relative_values(df_filtered, metrics_rel)
df_rel = df_filtered.copy()

In [73]:
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4723 entries, 0 to 7802
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64       
 1

# Function for densities

In [ ]:
# cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# # Create new columns by dividing by the 'Minutes' column
# for col in cols_mins:
#     df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [ ]:
# df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Find Max and Avg values before current date

In [54]:
def calculate_max_avg(df, metrics):

  # Create copies of the original DataFrame to ensure no unintended changes
  df = df.copy()

  # Loop through each metric to calculate max and avg values
  for metric in metrics:
      # Initialize the max and avg columns for the metric
      max_col = f"{metric}_max"
      avg_col = f"{metric}_avg"
      df[max_col] = None
      df[avg_col] = None

      # Iterate through each row to calculate max and avg based on previous dates
      for idx, row in df.iterrows():
          player_id = row['PlayerID']
          current_date = row['DATE']

          # Filter rows for the same player and only for dates before the current row's date
          player_data_before_date = df[
              (df['PlayerID'] == player_id) &
              (df['DATE'] < current_date)
          ]

          # Compute max and avg for the current metric based on the filtered data
          if not player_data_before_date.empty:
              df.at[idx, max_col] = player_data_before_date[metric].max()
              df.at[idx, avg_col] = player_data_before_date[metric].mean()
          else:
          # Fallback: Use max and avg across all rows for the same player
              player_data = df[df['PlayerID'] == player_id]
              df.at[idx, max_col] = player_data[metric].max()
              df.at[idx, avg_col] = player_data[metric].mean()

  return df

In [74]:
cols = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC','TD/Mins', 'HSR/Mins', '+25 Km/h/Mins', 'ACC/Mins', 'Sprints/Mins']
#df_max_avg = calculate_max_avg(df_rel, cols)
df_max_avg = df_rel.copy()

In [75]:
df_max_avg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4723 entries, 0 to 7802
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64       
 1

# Function for player loads

In [35]:
#df_max_avg

In [76]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    # Rolling sum, mean, std: exclude the current day using shift(1)
                    player_data[f'{col}-{dia}'] = (
                        player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                    )
                    # player_data[f'{col}-{dia}-avg'] = (
                    #     player_data[col].rolling(window=dia, min_periods=1).mean()
                    # )
                    # player_data[f'{col}-{dia}-std'] = (
                    #     player_data[col].rolling(window=dia, min_periods=1).std()
                    # )

                    # # EWMA: Exclude the current day using shift(1)
                    # player_data[f'{col}_EWMA-{dia}'] = (
                    #     player_data[col].ewm(span=dia, adjust=False).mean()
                    # )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [77]:
cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC', 'Sprints', 'Mins']

cumulative_df = calcular_acumulado(df_max_avg, cols_calculate, [3,7,21]) # loads -1, -3, -7, -21

In [78]:
cumulative_df.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21
0,2021-08-10,M-4,0.0,2021-2022,SEASON,5651.0,197.0,34.0,63.0,36.0,23085.0,CENTRE MIDFIELDER,28.41,19.0,80.35,30.73,20.278653,92.450374,40.098060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,CENTRE MIDFIELDER,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13


In [79]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64     

In [47]:
# cumulative_df = cumulative_df.dropna(subset=['TD-7', 'TD-7-std'])
# cumulative_df.reset_index(drop=True, inplace=True)

In [48]:
# cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 78 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4700 non-null   datetime64[ns]
 1   Microcycle          4485 non-null   object        
 2   Injury              4700 non-null   float64       
 3   preseason-season    4700 non-null   object        
 4   TD                  4700 non-null   float64       
 5   HSR                 4700 non-null   float64       
 6   +25 Km/h            4700 non-null   float64       
 7   ACC                 4700 non-null   float64       
 8   DEC                 4700 non-null   float64       
 9   PlayerID            4700 non-null   float64       
 10  Max Speed           4700 non-null   float64       
 11  Sprints             4700 non-null   float64       
 12  Mins                4700 non-null   float64       
 13  Max Speed Season    4700 non-null   float64     

# Calculate different load metrics

In [60]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-28-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

In [50]:
# cols_calculate = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']
# cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

In [51]:
# cumulative_df.drop(columns=['Sprints-28-avg', 'Sprints-28-std', 'DEC-28-avg', 'DEC-28-std', 'ACC-28-avg', 'ACC-28-std','+25 Km/h-28-avg', '+25 Km/h-28-std', 'HSR-28-avg', 'HSR-28-std', 'TD-28-avg', 'TD-28-std',
#                             'Sprints-7-avg', 'Sprints-7-std', 'DEC-7-avg', 'DEC-7-std', 'ACC-7-avg', 'ACC-7-std','+25 Km/h-7-avg', '+25 Km/h-7-std', 'HSR-7-avg', 'HSR-7-std', 'TD-7-avg', 'TD-7-std', 'TD-28', 'HSR-28', '+25 Km/h-28', 'ACC-28', 'DEC-28', 'Sprints-28',
#                             '+25 Km/h_EWMA-28', 'Sprints_EWMA-28', 'TD_EWMA-28', 'HSR_EWMA-28', 'ACC_EWMA-28', 'DEC_EWMA-28','Sprints_EWMA-7'] , inplace=True)

In [41]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   preseason-season    4723 non-null   object        
 4   TD                  4723 non-null   float64       
 5   >19.8               4723 non-null   float64       
 6   >25                 4723 non-null   float64       
 7   ACC                 4723 non-null   float64       
 8   DEC                 4723 non-null   float64       
 9   PlayerID            4723 non-null   float64       
 10  Player Position     4723 non-null   object        
 11  Max Speed           4723 non-null   float64       
 12  Sprints             4723 non-null   float64       
 13  Mins                4723 non-null   float64     

In [80]:
# Exclude 'Microcycle' column for counting NULLs
columns_to_check = cumulative_df.drop(columns=['Microcycle']).columns

# Drop rows where 2 or more null values exist (excluding 'Microcycle')
cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 1)]

# Display the resulting DataFrame
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4700 entries, 1 to 4722
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4700 non-null   datetime64[ns]
 1   Microcycle          4485 non-null   object        
 2   Injury              4700 non-null   float64       
 3   season              4700 non-null   object        
 4   preseason-season    4700 non-null   object        
 5   TD                  4700 non-null   float64       
 6   >19.8               4700 non-null   float64       
 7   >25                 4700 non-null   float64       
 8   ACC                 4700 non-null   float64       
 9   DEC                 4700 non-null   float64       
 10  PlayerID            4700 non-null   float64       
 11  Player Position     4700 non-null   object        
 12  Max Speed           4700 non-null   float64       
 13  Sprints             4700 non-null   float64       
 1

In [44]:
cumulative_df.columns

Index(['DATE', 'Microcycle', 'Injury', 'preseason-season', 'TD', '>19.8',
       '>25', 'ACC', 'DEC', 'PlayerID', 'Player Position', 'Max Speed',
       'Sprints', 'Mins', 'Max Speed Season', 'Avg Speed Season',
       '% Max Speed', 'Speed Diff Max Avg', 'TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', '>19.8-7', '>25-7',
       'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
       'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21'],
      dtype='object')

In [49]:
sasas = cumulative_df[['DATE','PlayerID','TD','TD-3','TD-7','TD-21']]
sasas

,DATE,PlayerID,TD,TD-3,TD-7,TD-21
1,2021-08-11,23085.0,7423.0,5651.0,5651.0,5651.0
2,2021-08-12,23085.0,1268.0,13074.0,13074.0,13074.0
3,2021-08-13,23085.0,4135.0,14342.0,14342.0,14342.0
4,2021-08-15,23085.0,6379.0,5403.0,18477.0,18477.0
5,2021-08-17,23085.0,6287.0,6379.0,24856.0,24856.0
...,...,...,...,...,...,...
4718,2022-05-16,42579.0,5436.0,6412.0,16067.0,46116.0
4719,2022-05-18,42579.0,4514.0,5436.0,17271.0,39887.0
4720,2022-05-19,42579.0,4959.0,9950.0,21785.0,44401.0
4721,2022-05-20,42579.0,2775.0,9473.0,21321.0,44353.0


Do KNN imputation for EMWA and ACWR values

In [ ]:
#cumulative_df.columns

In [ ]:
# # Exclude unwanted columns for KNN calculation
# columns_to_exclude = ['DATE', 'Microcycle', 'Injury', 'preseason-season']
# columns_for_knn = [col for col in cumulative_df.columns if col not in columns_to_exclude]

# # Apply KNN Imputation only to selected columns
# imputer = KNNImputer(n_neighbors=3)
# imputed_values = imputer.fit_transform(cumulative_df[columns_for_knn])

# # Replace imputed columns in the original dataframe
# cumulative_df[columns_for_knn] = imputed_values

# cumulative_df.info()

# Do the shift for 7 previous days injury

In [55]:
# def shift_injuries_7_days(df):
#   copy_df = df.copy()

#     # Initialize a new column for the 7-day injury prediction label
#   copy_df['Injury_7_day'] = 0

#   # Iterate over the df to assign labels to rows in the 7 days before an injury
#   for idx, row in copy_df.iterrows():
#       if row['Injury'] == 1:
#           # Get the player's ID
#           player_id = row['PlayerID']
#           injury_date = row['DATE']

#           # Assign injury label to the previous 7 days for the same player
#           # Change 0 to 1 to not include the day of the injury
#           for i in range(0, 8):
#               prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
#               mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
#               copy_df.loc[mask, 'Injury_7_day'] = 1

#   # Drop the original injury rows
#   # copy_df = copy_df[copy_df['Injury'] == 0]

#   return copy_df

In [81]:
#cumulative_df_inj = shift_injuries_7_days(cumulative_df)
cumulative_df_inj = cumulative_df.copy()

In [82]:
cumulative_df_inj

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,CENTRE MIDFIELDER,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,CENTRE MIDFIELDER,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4718,2022-05-16,M+1,0.0,2021-2022,SEASON,5436.0,284.0,41.0,69.0,57.0,42579.0,CENTRE BACK,28.31,22.0,71.47,29.76,19.989098,95.127688,41.627200,6412.0,24.0,0.0,102.0,74.0,4.0,138.71,16067.0,305.0,0.0,190.2,139.8,19.0,248.80,46116.0,1260.0,73.0,550.2,430.8,87.0,718.69
4719,2022-05-18,M-4,0.0,2021-2022,SEASON,4514.0,68.0,0.0,60.0,40.0,42579.0,CENTRE BACK,23.36,8.0,85.65,29.76,19.989098,78.494624,16.863701,5436.0,284.0,41.0,69.0,57.0,22.0,71.47,17271.0,383.0,41.0,214.0,166.0,35.0,281.15,39887.0,937.0,66.0,482.2,381.8,71.0,623.32
4720,2022-05-19,M-3,0.0,2021-2022,SEASON,4959.0,91.0,1.0,49.0,44.0,42579.0,CENTRE BACK,25.33,9.0,66.66,29.76,19.989098,85.114247,26.719073,9950.0,352.0,41.0,129.0,97.0,30.0,157.12,21785.0,451.0,41.0,274.0,206.0,43.0,366.80,44401.0,1005.0,66.0,542.2,421.8,79.0,708.97
4721,2022-05-20,M-2,0.0,2021-2022,SEASON,2775.0,1.0,0.0,49.0,24.0,42579.0,CENTRE BACK,19.89,0.0,55.09,29.76,19.989098,66.834677,-0.495761,9473.0,159.0,1.0,109.0,84.0,17.0,152.31,21321.0,467.0,42.0,280.0,215.0,43.0,362.49,44353.0,1048.0,56.0,539.2,412.8,82.0,708.96


In [83]:
print("No Injury: " + str(cumulative_df_inj[cumulative_df_inj["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df_inj[cumulative_df_inj["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df_inj[cumulative_df_inj['Injury'] == 1].shape[0] / cumulative_df_inj.shape[0]) * 100):.2f}%")

No Injury: 4663
Injury: 37
%: 0.79%


# Remove first 7 days of a player's data

In [84]:
# # Get the minimum date for each player
# min_dates = cumulative_df_inj.groupby('PlayerID')['DATE'].transform('min')

# # Calculate the cutoff date for each row (min date + 7 days)
# cutoff_dates = min_dates + pd.Timedelta(days=7)

# # Filter out rows where the date is within the first 7 days for each player
# final_df = cumulative_df_inj[cumulative_df_inj['DATE'] >= cutoff_dates]

final_df = cumulative_df_inj.copy()

In [85]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4700 entries, 1 to 4722
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4700 non-null   datetime64[ns]
 1   Microcycle          4485 non-null   object        
 2   Injury              4700 non-null   float64       
 3   season              4700 non-null   object        
 4   preseason-season    4700 non-null   object        
 5   TD                  4700 non-null   float64       
 6   >19.8               4700 non-null   float64       
 7   >25                 4700 non-null   float64       
 8   ACC                 4700 non-null   float64       
 9   DEC                 4700 non-null   float64       
 10  PlayerID            4700 non-null   float64       
 11  Player Position     4700 non-null   object        
 12  Max Speed           4700 non-null   float64       
 13  Sprints             4700 non-null   float64       
 1

In [86]:
print("No Injury: " + str(final_df[final_df["Injury_7_day"] == 0].shape[0]))
print("Injury: " + str(final_df[final_df["Injury_7_day"] == 1].shape[0]))
print(f"%: {((final_df[final_df['Injury_7_day'] == 1].shape[0] / final_df.shape[0]) * 100):.2f}%")

KeyError: 'Injury_7_day'

# Intensity metric based on Microcycle type

In [62]:
# final_df.Microcycle.unique()

array(['M-3', 'M-2', 'M-1', 'M+1', 'M-4', 'MD', 'M+3', 'M-5', 'M+2', nan,
       'MD(AWAY)', 'MD(HOME)'], dtype=object)

In [63]:
# # Map Microcycle to categories
# final_df['Category'] = final_df['Microcycle'].apply(
#     lambda x: 'MATCH' if x in ['MD', 'MD(AWAY)', 'MD(HOME)'] else 'TRAINING'
# )

# # Initialize the OneHotEncoder
# encoder = OneHotEncoder(sparse_output=False)

# # Fit and transform the categories
# encoded_categories = encoder.fit_transform(final_df[['Category']])

# # Add the encoded columns to the DataFrame with index alignment
# encoded_df = pd.DataFrame(
#     encoded_categories, 
#     columns=encoder.get_feature_names_out(['Category']), 
#     index=final_df.index  # Ensure index alignment
# )

# # Concatenate the original DataFrame and the one-hot encoded DataFrame
# encoded_final_df = pd.concat([final_df, encoded_df], axis=1)

In [87]:
final_df['Player Position'] = final_df['Player Position'].str.replace(" ", "")
final_df['Player Position'].unique()

array(['CENTREMIDFIELDER', 'STRIKER', 'WINGER', 'CENTREBACK', 'FULLBACK'],
      dtype=object)

In [88]:
# Example: One-hot encoding 'Player Position' column
encoded_final_df = pd.get_dummies(final_df, columns=['Player Position'], prefix='Position')

columns_to_convert = [
    'Position_CENTREBACK',
    'Position_CENTREMIDFIELDER',
    'Position_FULLBACK',
    'Position_STRIKER',
    'Position_WINGER'
]

# Convert the columns from bool to int
encoded_final_df[columns_to_convert] = encoded_final_df[columns_to_convert].astype(int)

print(encoded_final_df.dtypes)

# Display the resulting dataframe
encoded_final_df.head()

DATE                         datetime64[ns]
Microcycle                           object
Injury                              float64
season                               object
preseason-season                     object
TD                                  float64
>19.8                               float64
>25                                 float64
ACC                                 float64
DEC                                 float64
PlayerID                            float64
Max Speed                           float64
Sprints                             float64
Mins                                float64
Max Speed Season                    float64
Avg Speed Season                    float64
% Max Speed                         float64
Speed Diff Max Avg                  float64
TD-3                                float64
>19.8-3                             float64
>25-3                               float64
ACC-3                               float64
DEC-3                           

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,Position_CENTREBACK,Position_CENTREMIDFIELDER,Position_FULLBACK,Position_STRIKER,Position_WINGER
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,0,1,0,0,0
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,0,1,0,0,0
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,0,1,0,0,0
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,0,1,0,0,0
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,0,1,0,0,0


In [89]:
encoded_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4700 entries, 1 to 4722
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DATE                       4700 non-null   datetime64[ns]
 1   Microcycle                 4485 non-null   object        
 2   Injury                     4700 non-null   float64       
 3   season                     4700 non-null   object        
 4   preseason-season           4700 non-null   object        
 5   TD                         4700 non-null   float64       
 6   >19.8                      4700 non-null   float64       
 7   >25                        4700 non-null   float64       
 8   ACC                        4700 non-null   float64       
 9   DEC                        4700 non-null   float64       
 10  PlayerID                   4700 non-null   float64       
 11  Max Speed                  4700 non-null   float64       
 12  Sprints    

In [90]:
encoded_final_df[encoded_final_df['Injury']==1].head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,Position_CENTREBACK,Position_CENTREMIDFIELDER,Position_FULLBACK,Position_STRIKER,Position_WINGER
55,2022-01-08,MD,1.0,2021-2022,SEASON,4602.0,265.0,21.0,35.0,39.0,23085.0,29.37,22.0,36.34,30.73,20.278653,95.574357,44.832103,14181.0,328.0,28.0,89.0,64.0,25.0,179.16,25387.0,1187.0,73.0,241.0,142.0,62.0,304.80,44652.0,1739.0,146.0,388.0,236.0,93.0,587.80,0,1,0,0,0
243,2022-11-12,MD(AWAY),1.0,2022-2023,SEASON,9165.0,443.0,18.0,77.0,97.0,23085.0,31.99,40.0,80.47,32.12,20.304388,99.595268,57.552151,2963.0,27.0,0.0,12.0,15.0,3.0,40.27,23727.0,873.0,58.0,175.0,189.0,68.0,239.17,76951.0,2708.0,188.0,596.0,599.0,219.0,886.87,0,1,0,0,0
442,2022-12-06,M-5,1.0,2022-2023,SEASON,2547.0,14.0,0.0,16.0,27.0,87583.0,22.14,3.0,50.24,34.27,21.189939,64.604610,4.483546,15787.0,301.0,44.0,111.0,112.0,39.0,207.56,32465.0,1701.0,57.0,255.0,214.0,84.0,419.26,64465.0,2670.0,128.0,502.0,428.0,164.0,790.42,0,1,0,0,0
524,2022-12-11,MD(HOME),1.0,2022-2023,SEASON,2344.0,91.0,23.0,18.0,19.0,89871.0,29.08,2.0,22.08,34.12,21.753402,85.228605,33.680237,3396.0,66.0,13.0,25.0,20.0,1.0,69.98,20366.0,458.0,150.0,202.0,150.0,6.0,302.08,60412.0,1207.0,350.0,523.0,339.0,16.0,778.06,0,0,0,1,0
591,2023-01-07,MD(AWAY),1.0,2022-2023,SEASON,594.0,44.0,1.0,1.0,6.0,57090.0,26.01,0.0,5.35,33.86,20.742552,76.816302,25.394405,9137.0,42.0,0.0,75.0,61.0,0.0,149.42,21318.0,386.0,166.0,169.0,164.0,8.0,292.81,64849.0,1654.0,305.0,585.0,557.0,23.0,950.17,0,1,0,0,0


# Remove Preseason data

In [91]:

# Count of rows containing 0 for each column
zero_counts = (encoded_final_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Microcycle,0
2,Injury,4663
3,season,0
4,preseason-season,0
5,TD,0
6,>19.8,285
7,>25,1612
8,ACC,0
9,DEC,0


In [92]:
encoded_final_df.season.unique()

array(['2021-2022', '2022-2023'], dtype=object)

# Export data

In [107]:
encoded_final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy2.xlsx', index=False)

In [93]:
encoded_final_df.to_excel('data/cleaned_data_no_injured.xlsx', index=False)

In [ ]:
# result_df.to_csv('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy.csv', index=False)